In [42]:
BUG_REPORTS = "/home/joe0400/Downloads/eclipse_jdt.csv"
import html
import re
import spacy

nlp = spacy.load("en_core_web_sm")
def clean(text):
    # convert html escapes like &amp; to characters.
    text = html.unescape(text) 
    # tags like <tab>
    text = re.sub(r'<[^<>]*>', ' ', text)
    # markdown URLs like [Some text](https://....)
    text = re.sub(r'\[([^\[\]]*)\]\([^\(\)]*\)', r'\1', text)
    # text or code in brackets like [0]
    text = re.sub(r'\[[^\[\]]*\]', ' ', text)
    # standalone sequences of specials, matches &# but not #cool
    text = re.sub(r'(?:^|\s)[&#<>{}\[\]+|\\:-]{1,}(?:\s|$)', ' ', text)
    # standalone sequences of hyphens like --- or ==
    text = re.sub(r'(?:^|\s)[\-=\+]{2,}(?:\s|$)', ' ', text)
    # sequences of white spaces
    text = re.sub(r'\s+', ' ', text)
    return text.strip()
f = open(BUG_REPORTS, "r")
import csv

#read in the file, create the table, close the file, and seperate table header form the file
reader = csv.reader(f)
table = [[clean(e) for e in r] for r in reader]
f.close()

table_header = table[0]
table = table[1:]


In [43]:
import pandas
# this function gets the list of lemmas, joins them by space if they are
# nouns, proper nouns, adverbs, verbs, or adjectives
def get_nvas(string):
    return ' '.join([tok.lemma_ for tok in nlp(string) if tok.pos_ in ["PRON","NOUN","ADV","VERB","ADJ"]])
table_Frame = {name:[] for name in table_header}
table_Frame['words'] = []
#put the table elements in the table, and in the colum for words add the lemmatized version.
for each in table:
    for r, header in zip(each, table_header):
        table_Frame[header].append(r)
    table_Frame['words'].append(get_nvas(table_Frame['Description'][-1]))

df = pandas.DataFrame(table_Frame)

{'P1': 1122, 'P3': 39716, 'P2': 3063, 'P5': 252, 'P4': 1143}

In [45]:
#remove short bug reports
df = df[df['words'].str.len() > 50]

In [144]:
df = df[df['Duplicated_issue'] == '']
#removes duplicates

In [47]:
categories = [df[df["Priority"] == f"P{a}"] for a in range(1,6) if a != 3] + [df[df["Priority"] == "P3"].head(3000)]
#gets all ones that arent p3 for p3, it only gets the top 3000

In [49]:
df = pandas.concat(categories)
#construct a new table on this

In [51]:
df = df.sample(frac=1) # so that we can grab an random assortment, then i am going to grab 80%
#randomizes the order of teh table since now  its balanced more, i dont want to have inbalanced testing results

In [53]:
df_balanced = df

In [54]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df_balanced, test_size=0.2)
#creates the train test split

In [56]:
test # verification if it did what i think it did

,Issue_id,Priority,Component,Duplicated_issue,Title,Description,Status,Resolution,Version,Created_time,Resolved_time,words
1529,5114,P1,Debug,,Add exception dialog ignores caught/uncaught s...,Add an exception and note that the caught chec...,VERIFIED,FIXED,2.0,2001-10-19 13:19:00 -0400,2001-10-26 10:50:17 -0400,add exception note catch checkbox unchecked ex...
40065,280801,P2,Text,,Too many semicolons after constructor completion,I20090611-1540; ; public class Try {; Object m...,VERIFIED,FIXED,3.5,2009-06-18 12:47:00 -0400,2010-09-15 02:07:39 -0400,public class m return null select null type ne...
2780,7619,P1,Debug,,Walkback while debugging,Im not sure what I was doing when this error o...,VERIFIED,FIXED,2.0,2002-01-15 11:36:00 -0500,2002-01-21 12:12:13 -0500,I m sure what I do error occur I find the foll...
2173,6356,P3,UI,,Outliner has to delete actions,Product version: smoke test for integration bu...,VERIFIED,FIXED,2.0,2001-11-27 12:21:00 -0500,2002-01-16 02:33:06 -0500,product version smoke test integration build l...
827,3760,P3,UI,,JAR Packager should preserve timestamp from FS...,Currently the JAR packager sets the current ti...,VERIFIED,FIXED,2.0,2001-10-10 23:00:00 -0400,2001-10-24 07:19:24 -0400,currently packager set current time time stamp...
...,...,...,...,...,...,...,...,...,...,...,...,...
5571,14763,P3,UI,,(default package) doesnt go away,Create some classes without specifying a packa...,RESOLVED,WORKSFORME,2.0,2002-04-26 13:01:00 -0400,2002-05-30 18:50:09 -0400,create class specify package they appear the p...
26166,95679,P2,Debug,,Source not found,3.1M7; After installing 3.1M7 I noticed the de...,VERIFIED,FIXED,3.1,2005-05-17 18:36:00 -0400,2005-06-24 04:42:25 -0400,instal I notice debugger find source most proj...
1460,4982,P3,UI,,Automatic code assist must be configurable,Build 203.; There are cases where code assist ...,RESOLVED,FIXED,2.0,2001-10-15 11:00:00 -0400,2001-11-13 13:13:19 -0500,build there be case code assist pop free examp...
1122,4065,P3,UI,,ElementTreeSelectionDialog does not set focus ...,MA (25.07.2001 16:51:20); 1. Open the new Clas...,VERIFIED,FIXED,2.0,2001-10-10 23:05:00 -0400,2002-01-24 11:14:31 -0500,open new wizard folder input field press brows...


In [57]:
grid_param = [
    {
        'tfidf__min_df': [5, 10],
        'tfidf__ngram_range': [(1, 3), (1, 6)],
        'LinearSVC__penalty': ['l2'],
        'LinearSVC__loss': ['hinge'],
        'LinearSVC__max_iter': [10000]
    }, 
    {
        'tfidf__min_df': [5, 10],
        'tfidf__ngram_range': [(1, 3), (1, 6)],
        'LinearSVC__C': [1, 10],
        'LinearSVC__tol': [1e-2, 1e-3]
    }
]

In [62]:
len(list(train["words"]))

6238

In [ ]:
# does the linearsvc gridsearch

In [68]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
svc = Pipeline([("tfidf",TfidfVectorizer()),("LinearSVC",LinearSVC())])
clf = GridSearchCV(svc, grid_param)
encode_priority = {v:i for i,v in enumerate(set(train['Priority']))}
enumerated_priority = [encode_priority[v] for v in train["Priority"]]
clf.fit(list(train["words"]), enumerated_priority)

GridSearchCV(estimator=Pipeline(steps=[('tfidf', TfidfVectorizer()),
                                       ('LinearSVC', LinearSVC())]),
             param_grid=[{'LinearSVC__loss': ['hinge'],
                          'LinearSVC__max_iter': [10000],
                          'LinearSVC__penalty': ['l2'],
                          'tfidf__min_df': [5, 10],
                          'tfidf__ngram_range': [(1, 3), (1, 6)]},
                         {'LinearSVC__C': [1, 10],
                          'LinearSVC__tol': [0.01, 0.001],
                          'tfidf__min_df': [5, 10],
                          'tfidf__ngram_range': [(1, 3), (1, 6)]}])

In [130]:
from sklearn.svm import SVC
#does svc gridsearch
tuned_parameters = [{
'tfidf__min_df': [5, 10],
'tfidf__ngram_range': [(1, 3), (1, 6)],
"SVC__C": [1,10,100,1000],
"SVC__kernel": ['rbf', 'linear'],
"SVC__gamma": [1e-3, 1e-4],
"SVC__tol": [1e-2, 1e-10]
}]
svc2 = Pipeline([("tfidf",TfidfVectorizer()),("SVC",SVC())])
clf2 = GridSearchCV(svc2, tuned_parameters, verbose=4)
encode_priority = {v:i for i,v in enumerate(set(train['Priority']))}
enumerated_priority = [encode_priority[v] for v in train["Priority"]]
clf2.fit(list(train["words"]), enumerated_priority)

Fitting 5 folds for each of 128 candidates, totalling 640 fits
[CV 1/5] END SVC__C=1, SVC__gamma=0.001, SVC__kernel=rbf, SVC__tol=0.01, tfidf__min_df=5, tfidf__ngram_range=(1, 3);, score=0.379 total time=  13.5s
[CV 2/5] END SVC__C=1, SVC__gamma=0.001, SVC__kernel=rbf, SVC__tol=0.01, tfidf__min_df=5, tfidf__ngram_range=(1, 3);, score=0.379 total time=  14.1s
[CV 3/5] END SVC__C=1, SVC__gamma=0.001, SVC__kernel=rbf, SVC__tol=0.01, tfidf__min_df=5, tfidf__ngram_range=(1, 3);, score=0.379 total time=  13.5s


KeyboardInterrupt: 

In [74]:
svc_best = clf2.best_estimator_
#gets the best estimator for the svc

In [75]:
linear_svc_best = clf.best_estimator_
#gets linear svc best

In [76]:
svc_best.fit(list(train["words"]), enumerated_priority)
#train

Pipeline(steps=[('tfidf', TfidfVectorizer(min_df=5, ngram_range=(1, 6))),
                ('SVC', SVC(C=1, gamma=0.001, kernel='linear', tol=1e-10))])

In [77]:
linear_svc_best.fit(list(train["words"]), enumerated_priority)
#train

Pipeline(steps=[('tfidf', TfidfVectorizer(min_df=5, ngram_range=(1, 6))),
                ('LinearSVC', LinearSVC(loss='hinge', max_iter=10000))])

In [87]:
from sklearn.metrics import classification_report
enumerated_test = list(map(lambda a: encode_priority[a], test['Priority']))
print(classification_report(enumerated_test, linear_svc_best.predict(list(test["words"]))))
#get scores of the linear svc

              precision    recall  f1-score   support

           0       0.43      0.17      0.25       196
           1       0.51      0.65      0.57       635
           2       0.48      0.56      0.52       502
           3       0.00      0.00      0.00        36
           4       0.32      0.14      0.19       191

    accuracy                           0.48      1560
   macro avg       0.35      0.30      0.30      1560
weighted avg       0.45      0.48      0.45      1560



In [88]:
print(classification_report(enumerated_test, svc_best.predict(list(test["words"]))))
#get scores of the svc

              precision    recall  f1-score   support

           0       0.46      0.19      0.27       196
           1       0.50      0.69      0.58       635
           2       0.48      0.53      0.51       502
           3       0.00      0.00      0.00        36
           4       0.54      0.10      0.18       191

    accuracy                           0.49      1560
   macro avg       0.40      0.31      0.31      1560
weighted avg       0.48      0.49      0.46      1560



/home/joe0400/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/joe0400/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/joe0400/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [89]:
#i would say the svc is on average better than the svc, as its scores seem to be much higher on average
results = clf2.best_params_

In [90]:
results["SVC__kernel"] = "linear"

In [91]:
new_results = results
results["SVC__kernel"] = "rbf"
#this creates two pipelines to set parameters such that we can compare the two

In [93]:
svc2_copy = Pipeline([("tfidf",TfidfVectorizer()),("SVC",SVC())])
svc2_copy2 = Pipeline([("tfidf",TfidfVectorizer()),("SVC",SVC())])

In [95]:
svc2_copy.set_params(**results) #didnt know you had to unpack dicts for a kwargs, thought you could just pass it lol

Pipeline(steps=[('tfidf', TfidfVectorizer(min_df=5, ngram_range=(1, 6))),
                ('SVC', SVC(C=1, gamma=0.001, tol=1e-10))])

In [96]:
svc2_copy2.set_params(**new_results)

Pipeline(steps=[('tfidf', TfidfVectorizer(min_df=5, ngram_range=(1, 6))),
                ('SVC', SVC(C=1, gamma=0.001, tol=1e-10))])

In [98]:
svc2_copy.fit(list(train["words"]), enumerated_priority)
svc2_copy2.fit(list(train["words"]), enumerated_priority)

Pipeline(steps=[('tfidf', TfidfVectorizer(min_df=5, ngram_range=(1, 6))),
                ('SVC', SVC(C=1, gamma=0.001, tol=1e-10))])

In [99]:

print(classification_report(enumerated_test, svc2_copy.predict(list(test["words"]))))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       196
           1       0.41      1.00      0.58       635
           2       0.00      0.00      0.00       502
           3       0.00      0.00      0.00        36
           4       0.00      0.00      0.00       191

    accuracy                           0.41      1560
   macro avg       0.08      0.20      0.12      1560
weighted avg       0.17      0.41      0.24      1560



/home/joe0400/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/joe0400/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/joe0400/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [100]:
print(classification_report(enumerated_test, svc2_copy2.predict(list(test["words"]))))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       196
           1       0.41      1.00      0.58       635
           2       0.00      0.00      0.00       502
           3       0.00      0.00      0.00        36
           4       0.00      0.00      0.00       191

    accuracy                           0.41      1560
   macro avg       0.08      0.20      0.12      1560
weighted avg       0.17      0.41      0.24      1560



/home/joe0400/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/joe0400/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/joe0400/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [101]:
# both of these are god awful compared to the selected ones. due to that, 
# the original parameters were better for the svc.

In [102]:
reverse = {encode_priority[k]:k for k in encode_priority.keys()}
class_names = [encode_priority[k] for k in encode_priority.keys()]

In [103]:
!pip install lime
from lime.lime_text import LimeTextExplainer
explainer = LimeTextExplainer(class_names=class_names)

In [145]:
!pip install eli5
import eli5



In [133]:
#since i forgot to train the svc with probability=True, im rebuilding the pipeline with it, then training it
# since training it again with it is going to take around 10 hours, instead of 2.
# i know the best parameter so im going to train off of it.
params = svc_best.get_params()
params['SVC__probability'] = True
pipes = Pipeline([("tfidf",TfidfVectorizer()),("SVC",SVC(probability=True))])
pipes.set_params(**params)
pipes.fit(list(train["words"]), enumerated_priority)



{'memory': None, 'steps': [('tfidf', TfidfVectorizer(min_df=5, ngram_range=(1, 6))), ('SVC', SVC(C=1, gamma=0.001, kernel='linear', probability=True, tol=1e-10))], 'verbose': False, 'tfidf': TfidfVectorizer(min_df=5, ngram_range=(1, 6)), 'SVC': SVC(C=1, gamma=0.001, kernel='linear', probability=True, tol=1e-10), 'tfidf__analyzer': 'word', 'tfidf__binary': False, 'tfidf__decode_error': 'strict', 'tfidf__dtype': <class 'numpy.float64'>, 'tfidf__encoding': 'utf-8', 'tfidf__input': 'content', 'tfidf__lowercase': True, 'tfidf__max_df': 1.0, 'tfidf__max_features': None, 'tfidf__min_df': 5, 'tfidf__ngram_range': (1, 6), 'tfidf__norm': 'l2', 'tfidf__preprocessor': None, 'tfidf__smooth_idf': True, 'tfidf__stop_words': None, 'tfidf__strip_accents': None, 'tfidf__sublinear_tf': False, 'tfidf__token_pattern': '(?u)\\b\\w\\w+\\b', 'tfidf__tokenizer': None, 'tfidf__use_idf': True, 'tfidf__vocabulary': None, 'SVC__C': 1, 'SVC__break_ties': False, 'SVC__cache_size': 200, 'SVC__class_weight': None, 'SV

In [134]:
exp = explainer.explain_instance(list(test['words'])[0],pipes.predict_proba) 

In [139]:

print('Explanation for first test example')
print('\n'.join(map(str, exp.as_list())))
print()

Explanation for first test example
('breakpoint', -0.1867313745723174)
('note', 0.06931087291965311)
('uncaught', -0.05487712136475246)
('mark', 0.05020265688510582)
('viewer', 0.042097361339279125)
('add', 0.04168662900159278)
('catch', 0.04063441301158657)
('unchecked', -0.037377268342410064)
('show', 0.029218676695892704)
('checkbox', -0.015889339380631414)



In [141]:
# eli5.show_weights(pipes)
# i was reading up on this, it means that it can only explain 
# linearSVM ones
eli5.show_weights(linear_svc_best)

In [143]:
print(reverse.values())

dict_values(['P1', 'P3', 'P2', 'P5', 'P4'])


In [ ]:
# so for P1, compilation runtime, launchtarget, closed are prominent weight wise, 
# for p3, its note fix, and directory,
# for p2 its pull bug, fold, caret, select line are all important
# for p5 its box reproducable always, problem exists, and for 
# for p4 its note explorer, improve

# this suggest p1 is likele an issue on runtime, and seing breakpoint view, it looks like its a debugger issue
# for p3, it looks like its a file loading issue, as directories, package view, and stuff like that indicate that
# for p2 it looks like pull bug, fold, looks like its an issue with something related to version control
# for p5 it looks like it has some gui bug, one that makes it odd, and since it includes GTK, a toolkit by gnome
# for developing desktop applications, it suggests that.
# for p4, it looks like its  something integral to testing, and refactoring, meaning the project isnt in a clean
# more readable state.

# eli5 is awesome